## XGBoost

Notes:
    
1. Missing data should be set to 0. A unique feature of XGBoost is that it can handle missing data and simply
requires it to set to 0.

2. Only allows int, float and boolean data datatypes. So any categorical fields would need to converted.


In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from os import listdir
import re

import csv
import datetime
import pickle
import os

import xgboost as xgb

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

In [124]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score

In [3]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline, make_pipeline
from collections import Counter

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer


In [151]:
from sklearn import set_config
set_config(display='diagran')

In [5]:
pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)
pd.options.display.float_format = '{:.0f}'.format

In [6]:
DATA_ROOT_DIR='/mnt/data/projects/MD7'
PROJ_ROOT_DIR='/home/priyesh/projects/MD7'

In [7]:
filepath=os.path.join(PROJ_ROOT_DIR,'pickle','yahoo_complete_flat.pkl')
df_complete_flat = pd.read_pickle(filepath)

In [8]:
df_complete_flat.head()

,company,industry,0_inc_totalRevenue,0_inc_costOfRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netIncomeContinuousOperations,0_inc_netInterestIncome,0_inc_interestIncome,0_inc_otherIncomeExpense,0_inc_operatingExpense,0_inc_totalExpenses,0_inc_taxProvision,0_inc_interestExpense,0_inc_SGA,0_inc_researchAndDevelopment,0_inc_ebit,0_inc_dilutedEPS,0_inc_basicEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_currentAssets,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_financialAssets,0_bal_goodwill,0_bal_goodwillAndOtherIntangibleAssets,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_currentDebt,0_bal_payablesAndAccruedExpenses,0_bal_otherCurrentLiabilities,0_bal_currentLiabilities,0_bal_longTermDebt,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_totalNonCurrentLiabilities,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_cas_netIncome,0_cas_netIncomeFromContinuingOperations,0_cas_depreciation,0_cas_stockBasedCompensation,0_cas_cashFlowOperatingActivities,0_cas_capitalExpenditure,0_cas_saleOfBusiness,0_cas_purchaseOfBusiness,0_cas_netBusinessPurchaseAndSale,0_cas_saleOfPPE,0_cas_purchaseOfPPE,0_cas_netPPEPurchaseAndSale,0_cas_saleOfInvestment,0_cas_purchaseOfInvestment,0_cas_netInvestmentPurchaseAndSale,0_cas_saleOfInvestmentProperties,0_cas_purchaseOfInvestmentProperties,0_cas_netInvestmentPropertiesPurchaseAndSale,0_cas_saleOfIntangibles,0_cas_purchaseOfIntangibles,0_cas_netIntangiblesPurchaseAndSale,0_cas_netOtherInvestingChanges,0_cas_cashFlowInvestingActivities,0_cas_netIssuancePaymentsOfDebt,0_cas_netLongTermDebtIssuance,0_cas_netShortTermDebtIssuance,0_cas_commonStockDividendPaid,0_cas_preferredStockDividendPaid,0_cas_cashDividendsPaid,0_cas_netCommonStockIssuance,0_cas_netPreferredStockIssuance,0_cas_repurchaseOfCapitalStock,0_cas_netOtherFinancingCharges,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_totalRevenue,1_inc_costOfRevenue,1_inc_grossProfit,1_inc_netIncome,1_inc_operatingIncome,1_inc_netIncomeContinuousOperations,1_inc_netInterestIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_operatingExpense,1_inc_totalExpenses,1_inc_taxProvision,1_inc_interestExpense,1_inc_SGA,1_inc_researchAndDevelopment,1_inc_ebit,1_inc_dilutedEPS,1_inc_basicEPS,1_bal_cashEquivalent,1_bal_receivables,1_bal_finishedGoods,1_bal_workInProcess,1_bal_rawMaterials,1_bal_otherCurrentAssets,1_bal_inventory,1_bal_currentAssets,1_bal_netPPE,1_bal_otherNonCurrentAssets,1_bal_financialAssets,1_bal_goodwill,1_bal_goodwillAndOtherIntangibleAssets,1_bal_otherIntangibleAssets,1_bal_nonCurrentAccountsReceivable,1_bal_totalNonCurrentAssets,1_bal_currentDebt,1_bal_payablesAndAccruedExpenses,1_bal_otherCurrentLiabilities,1_bal_currentLiabilities,1_bal_longTermDebt,1_bal_otherNonCurrentLiabilities,1_bal_nonCurrentDeferredLiabilities,1_bal_nonCurrentDeferredTaxesLiabilities,1_bal_longTermProvisions,1_bal_totalNonCurrentLiabilities,1_bal_retainedEarnings,1_bal_stockholdersEquity,1_cas_netIncome,1_cas_netIncomeFromContinuingOperations,1_cas_depreciation,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_capitalExpenditure,1_cas_saleOfBusiness,1_cas_purchaseOfBusiness,1_cas_netBusinessPurchaseAndSale,1_cas_saleOfPPE,1_cas_purchaseOfPPE,1_cas_netPPEPurchaseAndSale,1_cas_saleOfInvestment,1_cas_purchaseOfInvestment,1_cas_netInvestmentPurchaseAndSale,1_cas_saleOfInvestmentProperties,1_cas_purchaseOfInvestmentProperties,1_cas_netInvestmentPropertiesPurchaseAndSale,1_cas_saleOfIntangibles,1_cas_purchaseOfIntangibles,1_cas_netIntangiblesPurchaseAndSale,1_cas_netOtherInvestingChanges,1_cas_cashFlowInvestingActivities,1_cas_netIssuancePaymentsOfDebt,1_cas_netLongTermDebtIssuance,1_cas_netShortTermDebtIssuance,1_cas_commonStockDividendPaid,1_cas_preferredStockDividendPaid,1_

In [9]:
df_complete_flat.iloc[:,0:10].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   company                              711 non-null    object
 1   industry                             711 non-null    object
 2   0_inc_totalRevenue                   711 non-null    int64 
 3   0_inc_costOfRevenue                  711 non-null    int64 
 4   0_inc_grossProfit                    711 non-null    int64 
 5   0_inc_netIncome                      711 non-null    int64 
 6   0_inc_operatingIncome                711 non-null    int64 
 7   0_inc_netIncomeContinuousOperations  711 non-null    int64 
 8   0_inc_netInterestIncome              711 non-null    int64 
 9   0_inc_interestIncome                 711 non-null    int64 
dtypes: int64(8), object(2)
memory usage: 55.7+ KB


In [19]:
# Take a copy so that we can always refer back to original 

df = df_complete_flat.copy()

In [22]:
df.head()

,company,industry,0_inc_totalRevenue,0_inc_costOfRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netIncomeContinuousOperations,0_inc_netInterestIncome,0_inc_interestIncome,0_inc_otherIncomeExpense,0_inc_operatingExpense,0_inc_totalExpenses,0_inc_taxProvision,0_inc_interestExpense,0_inc_SGA,0_inc_researchAndDevelopment,0_inc_ebit,0_inc_dilutedEPS,0_inc_basicEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_currentAssets,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_financialAssets,0_bal_goodwill,0_bal_goodwillAndOtherIntangibleAssets,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_currentDebt,0_bal_payablesAndAccruedExpenses,0_bal_otherCurrentLiabilities,0_bal_currentLiabilities,0_bal_longTermDebt,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_totalNonCurrentLiabilities,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_cas_netIncome,0_cas_netIncomeFromContinuingOperations,0_cas_depreciation,0_cas_stockBasedCompensation,0_cas_cashFlowOperatingActivities,0_cas_capitalExpenditure,0_cas_saleOfBusiness,0_cas_purchaseOfBusiness,0_cas_netBusinessPurchaseAndSale,0_cas_saleOfPPE,0_cas_purchaseOfPPE,0_cas_netPPEPurchaseAndSale,0_cas_saleOfInvestment,0_cas_purchaseOfInvestment,0_cas_netInvestmentPurchaseAndSale,0_cas_saleOfInvestmentProperties,0_cas_purchaseOfInvestmentProperties,0_cas_netInvestmentPropertiesPurchaseAndSale,0_cas_saleOfIntangibles,0_cas_purchaseOfIntangibles,0_cas_netIntangiblesPurchaseAndSale,0_cas_netOtherInvestingChanges,0_cas_cashFlowInvestingActivities,0_cas_netIssuancePaymentsOfDebt,0_cas_netLongTermDebtIssuance,0_cas_netShortTermDebtIssuance,0_cas_commonStockDividendPaid,0_cas_preferredStockDividendPaid,0_cas_cashDividendsPaid,0_cas_netCommonStockIssuance,0_cas_netPreferredStockIssuance,0_cas_repurchaseOfCapitalStock,0_cas_netOtherFinancingCharges,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_totalRevenue,1_inc_costOfRevenue,1_inc_grossProfit,1_inc_netIncome,1_inc_operatingIncome,1_inc_netIncomeContinuousOperations,1_inc_netInterestIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_operatingExpense,1_inc_totalExpenses,1_inc_taxProvision,1_inc_interestExpense,1_inc_SGA,1_inc_researchAndDevelopment,1_inc_ebit,1_inc_dilutedEPS,1_inc_basicEPS,1_bal_cashEquivalent,1_bal_receivables,1_bal_finishedGoods,1_bal_workInProcess,1_bal_rawMaterials,1_bal_otherCurrentAssets,1_bal_inventory,1_bal_currentAssets,1_bal_netPPE,1_bal_otherNonCurrentAssets,1_bal_financialAssets,1_bal_goodwill,1_bal_goodwillAndOtherIntangibleAssets,1_bal_otherIntangibleAssets,1_bal_nonCurrentAccountsReceivable,1_bal_totalNonCurrentAssets,1_bal_currentDebt,1_bal_payablesAndAccruedExpenses,1_bal_otherCurrentLiabilities,1_bal_currentLiabilities,1_bal_longTermDebt,1_bal_otherNonCurrentLiabilities,1_bal_nonCurrentDeferredLiabilities,1_bal_nonCurrentDeferredTaxesLiabilities,1_bal_longTermProvisions,1_bal_totalNonCurrentLiabilities,1_bal_retainedEarnings,1_bal_stockholdersEquity,1_cas_netIncome,1_cas_netIncomeFromContinuingOperations,1_cas_depreciation,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_capitalExpenditure,1_cas_saleOfBusiness,1_cas_purchaseOfBusiness,1_cas_netBusinessPurchaseAndSale,1_cas_saleOfPPE,1_cas_purchaseOfPPE,1_cas_netPPEPurchaseAndSale,1_cas_saleOfInvestment,1_cas_purchaseOfInvestment,1_cas_netInvestmentPurchaseAndSale,1_cas_saleOfInvestmentProperties,1_cas_purchaseOfInvestmentProperties,1_cas_netInvestmentPropertiesPurchaseAndSale,1_cas_saleOfIntangibles,1_cas_purchaseOfIntangibles,1_cas_netIntangiblesPurchaseAndSale,1_cas_netOtherInvestingChanges,1_cas_cashFlowInvestingActivities,1_cas_netIssuancePaymentsOfDebt,1_cas_netLongTermDebtIssuance,1_cas_netShortTermDebtIssuance,1_cas_commonStockDividendPaid,1_cas_preferredStockDividendPaid,1_

In [23]:
df.rename(columns={'FinalScore':'label'},inplace=True)

In [24]:
# Drop column company

df.drop('company',axis=1,inplace=True)

## Split Data into Independent and Dependent Variables

In [25]:
X = df.drop('label',axis=1)
X.head()

,industry,0_inc_totalRevenue,0_inc_costOfRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netIncomeContinuousOperations,0_inc_netInterestIncome,0_inc_interestIncome,0_inc_otherIncomeExpense,0_inc_operatingExpense,0_inc_totalExpenses,0_inc_taxProvision,0_inc_interestExpense,0_inc_SGA,0_inc_researchAndDevelopment,0_inc_ebit,0_inc_dilutedEPS,0_inc_basicEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_currentAssets,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_financialAssets,0_bal_goodwill,0_bal_goodwillAndOtherIntangibleAssets,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_currentDebt,0_bal_payablesAndAccruedExpenses,0_bal_otherCurrentLiabilities,0_bal_currentLiabilities,0_bal_longTermDebt,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_totalNonCurrentLiabilities,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_cas_netIncome,0_cas_netIncomeFromContinuingOperations,0_cas_depreciation,0_cas_stockBasedCompensation,0_cas_cashFlowOperatingActivities,0_cas_capitalExpenditure,0_cas_saleOfBusiness,0_cas_purchaseOfBusiness,0_cas_netBusinessPurchaseAndSale,0_cas_saleOfPPE,0_cas_purchaseOfPPE,0_cas_netPPEPurchaseAndSale,0_cas_saleOfInvestment,0_cas_purchaseOfInvestment,0_cas_netInvestmentPurchaseAndSale,0_cas_saleOfInvestmentProperties,0_cas_purchaseOfInvestmentProperties,0_cas_netInvestmentPropertiesPurchaseAndSale,0_cas_saleOfIntangibles,0_cas_purchaseOfIntangibles,0_cas_netIntangiblesPurchaseAndSale,0_cas_netOtherInvestingChanges,0_cas_cashFlowInvestingActivities,0_cas_netIssuancePaymentsOfDebt,0_cas_netLongTermDebtIssuance,0_cas_netShortTermDebtIssuance,0_cas_commonStockDividendPaid,0_cas_preferredStockDividendPaid,0_cas_cashDividendsPaid,0_cas_netCommonStockIssuance,0_cas_netPreferredStockIssuance,0_cas_repurchaseOfCapitalStock,0_cas_netOtherFinancingCharges,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_totalRevenue,1_inc_costOfRevenue,1_inc_grossProfit,1_inc_netIncome,1_inc_operatingIncome,1_inc_netIncomeContinuousOperations,1_inc_netInterestIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_operatingExpense,1_inc_totalExpenses,1_inc_taxProvision,1_inc_interestExpense,1_inc_SGA,1_inc_researchAndDevelopment,1_inc_ebit,1_inc_dilutedEPS,1_inc_basicEPS,1_bal_cashEquivalent,1_bal_receivables,1_bal_finishedGoods,1_bal_workInProcess,1_bal_rawMaterials,1_bal_otherCurrentAssets,1_bal_inventory,1_bal_currentAssets,1_bal_netPPE,1_bal_otherNonCurrentAssets,1_bal_financialAssets,1_bal_goodwill,1_bal_goodwillAndOtherIntangibleAssets,1_bal_otherIntangibleAssets,1_bal_nonCurrentAccountsReceivable,1_bal_totalNonCurrentAssets,1_bal_currentDebt,1_bal_payablesAndAccruedExpenses,1_bal_otherCurrentLiabilities,1_bal_currentLiabilities,1_bal_longTermDebt,1_bal_otherNonCurrentLiabilities,1_bal_nonCurrentDeferredLiabilities,1_bal_nonCurrentDeferredTaxesLiabilities,1_bal_longTermProvisions,1_bal_totalNonCurrentLiabilities,1_bal_retainedEarnings,1_bal_stockholdersEquity,1_cas_netIncome,1_cas_netIncomeFromContinuingOperations,1_cas_depreciation,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_capitalExpenditure,1_cas_saleOfBusiness,1_cas_purchaseOfBusiness,1_cas_netBusinessPurchaseAndSale,1_cas_saleOfPPE,1_cas_purchaseOfPPE,1_cas_netPPEPurchaseAndSale,1_cas_saleOfInvestment,1_cas_purchaseOfInvestment,1_cas_netInvestmentPurchaseAndSale,1_cas_saleOfInvestmentProperties,1_cas_purchaseOfInvestmentProperties,1_cas_netInvestmentPropertiesPurchaseAndSale,1_cas_saleOfIntangibles,1_cas_purchaseOfIntangibles,1_cas_netIntangiblesPurchaseAndSale,1_cas_netOtherInvestingChanges,1_cas_cashFlowInvestingActivities,1_cas_netIssuancePaymentsOfDebt,1_cas_netLongTermDebtIssuance,1_cas_netShortTermDebtIssuance,1_cas_commonStockDividendPaid,1_cas_preferredStockDividendPaid,1_cas_cash

In [26]:
y = df['label']
y.head()

0    3
1    2
2    0
3    2
4    2
Name: label, dtype: int64

In [27]:
X.head()

,industry,0_inc_totalRevenue,0_inc_costOfRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netIncomeContinuousOperations,0_inc_netInterestIncome,0_inc_interestIncome,0_inc_otherIncomeExpense,0_inc_operatingExpense,0_inc_totalExpenses,0_inc_taxProvision,0_inc_interestExpense,0_inc_SGA,0_inc_researchAndDevelopment,0_inc_ebit,0_inc_dilutedEPS,0_inc_basicEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_currentAssets,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_financialAssets,0_bal_goodwill,0_bal_goodwillAndOtherIntangibleAssets,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_currentDebt,0_bal_payablesAndAccruedExpenses,0_bal_otherCurrentLiabilities,0_bal_currentLiabilities,0_bal_longTermDebt,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_totalNonCurrentLiabilities,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_cas_netIncome,0_cas_netIncomeFromContinuingOperations,0_cas_depreciation,0_cas_stockBasedCompensation,0_cas_cashFlowOperatingActivities,0_cas_capitalExpenditure,0_cas_saleOfBusiness,0_cas_purchaseOfBusiness,0_cas_netBusinessPurchaseAndSale,0_cas_saleOfPPE,0_cas_purchaseOfPPE,0_cas_netPPEPurchaseAndSale,0_cas_saleOfInvestment,0_cas_purchaseOfInvestment,0_cas_netInvestmentPurchaseAndSale,0_cas_saleOfInvestmentProperties,0_cas_purchaseOfInvestmentProperties,0_cas_netInvestmentPropertiesPurchaseAndSale,0_cas_saleOfIntangibles,0_cas_purchaseOfIntangibles,0_cas_netIntangiblesPurchaseAndSale,0_cas_netOtherInvestingChanges,0_cas_cashFlowInvestingActivities,0_cas_netIssuancePaymentsOfDebt,0_cas_netLongTermDebtIssuance,0_cas_netShortTermDebtIssuance,0_cas_commonStockDividendPaid,0_cas_preferredStockDividendPaid,0_cas_cashDividendsPaid,0_cas_netCommonStockIssuance,0_cas_netPreferredStockIssuance,0_cas_repurchaseOfCapitalStock,0_cas_netOtherFinancingCharges,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_totalRevenue,1_inc_costOfRevenue,1_inc_grossProfit,1_inc_netIncome,1_inc_operatingIncome,1_inc_netIncomeContinuousOperations,1_inc_netInterestIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_operatingExpense,1_inc_totalExpenses,1_inc_taxProvision,1_inc_interestExpense,1_inc_SGA,1_inc_researchAndDevelopment,1_inc_ebit,1_inc_dilutedEPS,1_inc_basicEPS,1_bal_cashEquivalent,1_bal_receivables,1_bal_finishedGoods,1_bal_workInProcess,1_bal_rawMaterials,1_bal_otherCurrentAssets,1_bal_inventory,1_bal_currentAssets,1_bal_netPPE,1_bal_otherNonCurrentAssets,1_bal_financialAssets,1_bal_goodwill,1_bal_goodwillAndOtherIntangibleAssets,1_bal_otherIntangibleAssets,1_bal_nonCurrentAccountsReceivable,1_bal_totalNonCurrentAssets,1_bal_currentDebt,1_bal_payablesAndAccruedExpenses,1_bal_otherCurrentLiabilities,1_bal_currentLiabilities,1_bal_longTermDebt,1_bal_otherNonCurrentLiabilities,1_bal_nonCurrentDeferredLiabilities,1_bal_nonCurrentDeferredTaxesLiabilities,1_bal_longTermProvisions,1_bal_totalNonCurrentLiabilities,1_bal_retainedEarnings,1_bal_stockholdersEquity,1_cas_netIncome,1_cas_netIncomeFromContinuingOperations,1_cas_depreciation,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_capitalExpenditure,1_cas_saleOfBusiness,1_cas_purchaseOfBusiness,1_cas_netBusinessPurchaseAndSale,1_cas_saleOfPPE,1_cas_purchaseOfPPE,1_cas_netPPEPurchaseAndSale,1_cas_saleOfInvestment,1_cas_purchaseOfInvestment,1_cas_netInvestmentPurchaseAndSale,1_cas_saleOfInvestmentProperties,1_cas_purchaseOfInvestmentProperties,1_cas_netInvestmentPropertiesPurchaseAndSale,1_cas_saleOfIntangibles,1_cas_purchaseOfIntangibles,1_cas_netIntangiblesPurchaseAndSale,1_cas_netOtherInvestingChanges,1_cas_cashFlowInvestingActivities,1_cas_netIssuancePaymentsOfDebt,1_cas_netLongTermDebtIssuance,1_cas_netShortTermDebtIssuance,1_cas_commonStockDividendPaid,1_cas_preferredStockDividendPaid,1_cas_cash

## Train Test Split

In [28]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

## Column Transformer

In [38]:
numeric_cols = [feature for feature in df.columns if df[feature].dtype != 'O']
df[numeric_cols].dtypes

0_inc_totalRevenue                              int64
0_inc_costOfRevenue                             int64
0_inc_grossProfit                               int64
0_inc_netIncome                                 int64
0_inc_operatingIncome                           int64
0_inc_netIncomeContinuousOperations             int64
0_inc_netInterestIncome                         int64
0_inc_interestIncome                            int64
0_inc_otherIncomeExpense                        int64
0_inc_operatingExpense                          int64
0_inc_totalExpenses                             int64
0_inc_taxProvision                              int64
0_inc_interestExpense                           int64
0_inc_SGA                                       int64
0_inc_researchAndDevelopment                    int64
0_inc_ebit                                      int64
0_inc_dilutedEPS                                int64
0_inc_basicEPS                                  int64
0_bal_cashEquivalent        

In [47]:
# Create Transformer

transformer = ColumnTransformer(transformers=[
      ('tnf1',SimpleImputer(missing_values = np.nan,strategy='constant',fill_value=0),['3_cas_freeCashFlow']), 
      ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['industry']) 
],remainder='passthrough')

In [53]:
X_temp = transformer.fit_transform(X_train)
X_temp

array([[ 1.55000e+09,  0.00000e+00,  0.00000e+00, ..., -6.47000e+08,
        -1.53000e+08, -1.56200e+09],
       [ 5.83610e+08,  0.00000e+00,  0.00000e+00, ..., -7.28655e+08,
         0.00000e+00, -9.38781e+08],
       [ 7.01530e+08,  0.00000e+00,  0.00000e+00, ..., -1.27762e+08,
        -9.42750e+07, -3.11071e+08],
       ...,
       [ 7.67200e+08,  0.00000e+00,  0.00000e+00, ..., -2.37800e+08,
         0.00000e+00, -7.74900e+08],
       [ 4.06162e+08,  0.00000e+00,  0.00000e+00, ..., -3.38880e+08,
        -1.69093e+08, -5.26848e+08],
       [ 3.40500e+09,  0.00000e+00,  0.00000e+00, ..., -1.50000e+09,
         7.23000e+08, -6.89000e+08]])

In [52]:
transformer.fit_transform(X_test).shape

(143, 388)

In [11]:
# Drop column company

df.drop('company',axis=1,inplace=True)

In [ ]:
# Replace space in industry column with _

df['industry'].replace(' ','_',regex=True, inplace=True)
df.head()

In [ ]:
df.rename(columns={'FinalScore':'label'},inplace=True)

In [ ]:
df.head()

In [ ]:
df['label'].value_counts().plot(kind='bar')

In [ ]:
X = df.drop('label',axis=1)
X.head()

In [ ]:
df['label'].value_counts()

In [ ]:
y = df['label']
y.head()

## One hot encoding

In [30]:
X_encoded = pd.get_dummies(X, columns=['industry'])

In [32]:
X_encoded.head()

,0_inc_totalRevenue,0_inc_costOfRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netIncomeContinuousOperations,0_inc_netInterestIncome,0_inc_interestIncome,0_inc_otherIncomeExpense,0_inc_operatingExpense,0_inc_totalExpenses,0_inc_taxProvision,0_inc_interestExpense,0_inc_SGA,0_inc_researchAndDevelopment,0_inc_ebit,0_inc_dilutedEPS,0_inc_basicEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_currentAssets,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_financialAssets,0_bal_goodwill,0_bal_goodwillAndOtherIntangibleAssets,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_currentDebt,0_bal_payablesAndAccruedExpenses,0_bal_otherCurrentLiabilities,0_bal_currentLiabilities,0_bal_longTermDebt,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_totalNonCurrentLiabilities,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_cas_netIncome,0_cas_netIncomeFromContinuingOperations,0_cas_depreciation,0_cas_stockBasedCompensation,0_cas_cashFlowOperatingActivities,0_cas_capitalExpenditure,0_cas_saleOfBusiness,0_cas_purchaseOfBusiness,0_cas_netBusinessPurchaseAndSale,0_cas_saleOfPPE,0_cas_purchaseOfPPE,0_cas_netPPEPurchaseAndSale,0_cas_saleOfInvestment,0_cas_purchaseOfInvestment,0_cas_netInvestmentPurchaseAndSale,0_cas_saleOfInvestmentProperties,0_cas_purchaseOfInvestmentProperties,0_cas_netInvestmentPropertiesPurchaseAndSale,0_cas_saleOfIntangibles,0_cas_purchaseOfIntangibles,0_cas_netIntangiblesPurchaseAndSale,0_cas_netOtherInvestingChanges,0_cas_cashFlowInvestingActivities,0_cas_netIssuancePaymentsOfDebt,0_cas_netLongTermDebtIssuance,0_cas_netShortTermDebtIssuance,0_cas_commonStockDividendPaid,0_cas_preferredStockDividendPaid,0_cas_cashDividendsPaid,0_cas_netCommonStockIssuance,0_cas_netPreferredStockIssuance,0_cas_repurchaseOfCapitalStock,0_cas_netOtherFinancingCharges,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_totalRevenue,1_inc_costOfRevenue,1_inc_grossProfit,1_inc_netIncome,1_inc_operatingIncome,1_inc_netIncomeContinuousOperations,1_inc_netInterestIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_operatingExpense,1_inc_totalExpenses,1_inc_taxProvision,1_inc_interestExpense,1_inc_SGA,1_inc_researchAndDevelopment,1_inc_ebit,1_inc_dilutedEPS,1_inc_basicEPS,1_bal_cashEquivalent,1_bal_receivables,1_bal_finishedGoods,1_bal_workInProcess,1_bal_rawMaterials,1_bal_otherCurrentAssets,1_bal_inventory,1_bal_currentAssets,1_bal_netPPE,1_bal_otherNonCurrentAssets,1_bal_financialAssets,1_bal_goodwill,1_bal_goodwillAndOtherIntangibleAssets,1_bal_otherIntangibleAssets,1_bal_nonCurrentAccountsReceivable,1_bal_totalNonCurrentAssets,1_bal_currentDebt,1_bal_payablesAndAccruedExpenses,1_bal_otherCurrentLiabilities,1_bal_currentLiabilities,1_bal_longTermDebt,1_bal_otherNonCurrentLiabilities,1_bal_nonCurrentDeferredLiabilities,1_bal_nonCurrentDeferredTaxesLiabilities,1_bal_longTermProvisions,1_bal_totalNonCurrentLiabilities,1_bal_retainedEarnings,1_bal_stockholdersEquity,1_cas_netIncome,1_cas_netIncomeFromContinuingOperations,1_cas_depreciation,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_capitalExpenditure,1_cas_saleOfBusiness,1_cas_purchaseOfBusiness,1_cas_netBusinessPurchaseAndSale,1_cas_saleOfPPE,1_cas_purchaseOfPPE,1_cas_netPPEPurchaseAndSale,1_cas_saleOfInvestment,1_cas_purchaseOfInvestment,1_cas_netInvestmentPurchaseAndSale,1_cas_saleOfInvestmentProperties,1_cas_purchaseOfInvestmentProperties,1_cas_netInvestmentPropertiesPurchaseAndSale,1_cas_saleOfIntangibles,1_cas_purchaseOfIntangibles,1_cas_netIntangiblesPurchaseAndSale,1_cas_netOtherInvestingChanges,1_cas_cashFlowInvestingActivities,1_cas_netIssuancePaymentsOfDebt,1_cas_netLongTermDebtIssuance,1_cas_netShortTermDebtIssuance,1_cas_commonStockDividendPaid,1_cas_preferredStockDividendPaid,1_cas_cashDividends

In [31]:
oversample = SMOTE()
X_encoded, y = oversample.fit_resample(X_encoded,y)

In [33]:
oversample = SMOTE()
X_temp, y_temp = oversample.fit_resample(X_encoded,y)

In [34]:
X_encoded.head()

,0_inc_totalRevenue,0_inc_costOfRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netIncomeContinuousOperations,0_inc_netInterestIncome,0_inc_interestIncome,0_inc_otherIncomeExpense,0_inc_operatingExpense,0_inc_totalExpenses,0_inc_taxProvision,0_inc_interestExpense,0_inc_SGA,0_inc_researchAndDevelopment,0_inc_ebit,0_inc_dilutedEPS,0_inc_basicEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_currentAssets,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_financialAssets,0_bal_goodwill,0_bal_goodwillAndOtherIntangibleAssets,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_currentDebt,0_bal_payablesAndAccruedExpenses,0_bal_otherCurrentLiabilities,0_bal_currentLiabilities,0_bal_longTermDebt,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_totalNonCurrentLiabilities,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_cas_netIncome,0_cas_netIncomeFromContinuingOperations,0_cas_depreciation,0_cas_stockBasedCompensation,0_cas_cashFlowOperatingActivities,0_cas_capitalExpenditure,0_cas_saleOfBusiness,0_cas_purchaseOfBusiness,0_cas_netBusinessPurchaseAndSale,0_cas_saleOfPPE,0_cas_purchaseOfPPE,0_cas_netPPEPurchaseAndSale,0_cas_saleOfInvestment,0_cas_purchaseOfInvestment,0_cas_netInvestmentPurchaseAndSale,0_cas_saleOfInvestmentProperties,0_cas_purchaseOfInvestmentProperties,0_cas_netInvestmentPropertiesPurchaseAndSale,0_cas_saleOfIntangibles,0_cas_purchaseOfIntangibles,0_cas_netIntangiblesPurchaseAndSale,0_cas_netOtherInvestingChanges,0_cas_cashFlowInvestingActivities,0_cas_netIssuancePaymentsOfDebt,0_cas_netLongTermDebtIssuance,0_cas_netShortTermDebtIssuance,0_cas_commonStockDividendPaid,0_cas_preferredStockDividendPaid,0_cas_cashDividendsPaid,0_cas_netCommonStockIssuance,0_cas_netPreferredStockIssuance,0_cas_repurchaseOfCapitalStock,0_cas_netOtherFinancingCharges,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_totalRevenue,1_inc_costOfRevenue,1_inc_grossProfit,1_inc_netIncome,1_inc_operatingIncome,1_inc_netIncomeContinuousOperations,1_inc_netInterestIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_operatingExpense,1_inc_totalExpenses,1_inc_taxProvision,1_inc_interestExpense,1_inc_SGA,1_inc_researchAndDevelopment,1_inc_ebit,1_inc_dilutedEPS,1_inc_basicEPS,1_bal_cashEquivalent,1_bal_receivables,1_bal_finishedGoods,1_bal_workInProcess,1_bal_rawMaterials,1_bal_otherCurrentAssets,1_bal_inventory,1_bal_currentAssets,1_bal_netPPE,1_bal_otherNonCurrentAssets,1_bal_financialAssets,1_bal_goodwill,1_bal_goodwillAndOtherIntangibleAssets,1_bal_otherIntangibleAssets,1_bal_nonCurrentAccountsReceivable,1_bal_totalNonCurrentAssets,1_bal_currentDebt,1_bal_payablesAndAccruedExpenses,1_bal_otherCurrentLiabilities,1_bal_currentLiabilities,1_bal_longTermDebt,1_bal_otherNonCurrentLiabilities,1_bal_nonCurrentDeferredLiabilities,1_bal_nonCurrentDeferredTaxesLiabilities,1_bal_longTermProvisions,1_bal_totalNonCurrentLiabilities,1_bal_retainedEarnings,1_bal_stockholdersEquity,1_cas_netIncome,1_cas_netIncomeFromContinuingOperations,1_cas_depreciation,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_capitalExpenditure,1_cas_saleOfBusiness,1_cas_purchaseOfBusiness,1_cas_netBusinessPurchaseAndSale,1_cas_saleOfPPE,1_cas_purchaseOfPPE,1_cas_netPPEPurchaseAndSale,1_cas_saleOfInvestment,1_cas_purchaseOfInvestment,1_cas_netInvestmentPurchaseAndSale,1_cas_saleOfInvestmentProperties,1_cas_purchaseOfInvestmentProperties,1_cas_netInvestmentPropertiesPurchaseAndSale,1_cas_saleOfIntangibles,1_cas_purchaseOfIntangibles,1_cas_netIntangiblesPurchaseAndSale,1_cas_netOtherInvestingChanges,1_cas_cashFlowInvestingActivities,1_cas_netIssuancePaymentsOfDebt,1_cas_netLongTermDebtIssuance,1_cas_netShortTermDebtIssuance,1_cas_commonStockDividendPaid,1_cas_preferredStockDividendPaid,1_cas_cashDividends

In [ ]:
counter = Counter(y)
for k, v in counter.items():
  dist = v / len(y) * 100
  print(f"Class={k}, n={v} ({dist}%)")

In [ ]:
oversample = SMOTE()
undersample = RandomUnderSampler()
#steps = [("o", oversample),("u",undersample)]
steps = [("o", oversample)]
pipeline = Pipeline(steps=steps)
# transform the dataset 
X_encoded, y = pipeline.fit_resample(X_encoded,y)

In [ ]:
#model = xgb.XGBClassifier(random_state=666)
#scores = cross_validate(model, X_encoded, y, cv=6, n_jobs=-1, scoring="f1_macro")
#scores

In [ ]:
XGB_dataset = {}
XGB_dataset['X'] = X_encoded
XGB_dataset['y'] = y

In [ ]:
## Save Data

filepath=os.path.join(PROJ_ROOT_DIR,'pickle','XGB_dataset.pkl')
with open(filepath,'wb') as f:
    pickle.dump(XGB_dataset,f)

In [ ]:
XGB_dataset['X']

In [ ]:
# Instantiate XGBClassifier

xgb_clf = xgb.XGBClassifier(random_state=666)

#Inspect the parameters

xgb_clf.get_params()

In [ ]:
# Set parameters 

xgb_clf.set_params(n_estimators=10)

xgb_clf.set_params(max_depth=3)

xgb_clf.set_params(eval_metric='error')



In [ ]:
# Fit to training set

xgb_clf.fit(X_train, y_train)


In [ ]:
# Predict the labels of the test set

preds = xgb_clf.predict(X_test)


In [ ]:
# Calculate the accuracy

accuracy = float(np.sum(preds==y_test))/y_test.shape[0]

accuracy

In [ ]:
## Plot Feature importance
import matplotlib

matplotlib.rcParams['figure.figsize'] = (10.0,8)

# Plot feature importance

xgb.plot_importance(xgb_clf)

In [ ]:
## Plot a decision tree from model

matplotlib.rcParams['figure.figsize'] = (20.0,8)

# Plot the first tree

xgb.plot_tree(xgb_clf,num_trees=0)

In [ ]:
# Try with different parameters

xgb_clf.set_params(n_estimators=100)

xgb_clf.set_params(max_depth=3)

# Fit to training set

xgb_clf.fit(X_train, y_train)

# Predict the labels of the test set

preds = xgb_clf.predict(X_test)

# Calculate the accuracy

accuracy = float(np.sum(preds==y_test))/y_test.shape[0]

accuracy


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define a parameter grid

rs_param_grid = {
    # max_depth: values from 3 to 12
    
    'max_depth': [3,4,5,6,7,8,9,10,11,12],
    
    # alpha: values 0, .001, .01, .1
    
    'alpha': [0, .001, .01, .1],
    
    # subsample: values 0.25, 0.5, 0.75, 1
    
    'subsample': [0.25,0.5,0.75,1],
    
    # learning_rate: ten values between 0.01 = 0.5
    
    'learning_rate': np.linspace(0.01, 0.5, 10),
    
    # n_estimators: values 10, 25, 40
    
    'n_estimators':[30,40,50,60,70,80,90,100]
}


In [ ]:
# Instantiate XGBoost Classifier

xgb_clf = xgb.XGBClassifier(eval_metric='error',random_state=123)

In [ ]:
#Instantiate RandomizedSearch

xgb_rs = RandomizedSearchCV(estimator=xgb_clf, 
                            param_distributions=rs_param_grid, cv=5, n_iter=200, verbose=1, random_state=666)

In [ ]:
# Train the mode on the training set

xgb_rs.fit(X_train, y_train)

In [ ]:
# Print the best parameters and highest accuracy

print("Best parameters found:", xgb_rs.best_params_)
print("Best accuracy found:", xgb_rs.best_score_)

In [ ]:
clf_xgb = xgb.XGBClassifier(objective='multi:softmax',num_class=4,max_depth=6, seed=666,early_stopping_rounds=10 )

clf_xgb.fit(X_train, y_train, verbose=True, 
            eval_metric='aucpr',
            early_stopping_rounds=10,
            eval_set=[(X_test, y_test)])

# Pipelines

In [ ]:
Steps:
* Missing Values
* One hot encoding
* Scaling
* feature selection
* Train model

In [55]:
df.head()

,industry,0_inc_totalRevenue,0_inc_costOfRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netIncomeContinuousOperations,0_inc_netInterestIncome,0_inc_interestIncome,0_inc_otherIncomeExpense,0_inc_operatingExpense,0_inc_totalExpenses,0_inc_taxProvision,0_inc_interestExpense,0_inc_SGA,0_inc_researchAndDevelopment,0_inc_ebit,0_inc_dilutedEPS,0_inc_basicEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_currentAssets,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_financialAssets,0_bal_goodwill,0_bal_goodwillAndOtherIntangibleAssets,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_currentDebt,0_bal_payablesAndAccruedExpenses,0_bal_otherCurrentLiabilities,0_bal_currentLiabilities,0_bal_longTermDebt,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_totalNonCurrentLiabilities,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_cas_netIncome,0_cas_netIncomeFromContinuingOperations,0_cas_depreciation,0_cas_stockBasedCompensation,0_cas_cashFlowOperatingActivities,0_cas_capitalExpenditure,0_cas_saleOfBusiness,0_cas_purchaseOfBusiness,0_cas_netBusinessPurchaseAndSale,0_cas_saleOfPPE,0_cas_purchaseOfPPE,0_cas_netPPEPurchaseAndSale,0_cas_saleOfInvestment,0_cas_purchaseOfInvestment,0_cas_netInvestmentPurchaseAndSale,0_cas_saleOfInvestmentProperties,0_cas_purchaseOfInvestmentProperties,0_cas_netInvestmentPropertiesPurchaseAndSale,0_cas_saleOfIntangibles,0_cas_purchaseOfIntangibles,0_cas_netIntangiblesPurchaseAndSale,0_cas_netOtherInvestingChanges,0_cas_cashFlowInvestingActivities,0_cas_netIssuancePaymentsOfDebt,0_cas_netLongTermDebtIssuance,0_cas_netShortTermDebtIssuance,0_cas_commonStockDividendPaid,0_cas_preferredStockDividendPaid,0_cas_cashDividendsPaid,0_cas_netCommonStockIssuance,0_cas_netPreferredStockIssuance,0_cas_repurchaseOfCapitalStock,0_cas_netOtherFinancingCharges,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_totalRevenue,1_inc_costOfRevenue,1_inc_grossProfit,1_inc_netIncome,1_inc_operatingIncome,1_inc_netIncomeContinuousOperations,1_inc_netInterestIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_operatingExpense,1_inc_totalExpenses,1_inc_taxProvision,1_inc_interestExpense,1_inc_SGA,1_inc_researchAndDevelopment,1_inc_ebit,1_inc_dilutedEPS,1_inc_basicEPS,1_bal_cashEquivalent,1_bal_receivables,1_bal_finishedGoods,1_bal_workInProcess,1_bal_rawMaterials,1_bal_otherCurrentAssets,1_bal_inventory,1_bal_currentAssets,1_bal_netPPE,1_bal_otherNonCurrentAssets,1_bal_financialAssets,1_bal_goodwill,1_bal_goodwillAndOtherIntangibleAssets,1_bal_otherIntangibleAssets,1_bal_nonCurrentAccountsReceivable,1_bal_totalNonCurrentAssets,1_bal_currentDebt,1_bal_payablesAndAccruedExpenses,1_bal_otherCurrentLiabilities,1_bal_currentLiabilities,1_bal_longTermDebt,1_bal_otherNonCurrentLiabilities,1_bal_nonCurrentDeferredLiabilities,1_bal_nonCurrentDeferredTaxesLiabilities,1_bal_longTermProvisions,1_bal_totalNonCurrentLiabilities,1_bal_retainedEarnings,1_bal_stockholdersEquity,1_cas_netIncome,1_cas_netIncomeFromContinuingOperations,1_cas_depreciation,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_capitalExpenditure,1_cas_saleOfBusiness,1_cas_purchaseOfBusiness,1_cas_netBusinessPurchaseAndSale,1_cas_saleOfPPE,1_cas_purchaseOfPPE,1_cas_netPPEPurchaseAndSale,1_cas_saleOfInvestment,1_cas_purchaseOfInvestment,1_cas_netInvestmentPurchaseAndSale,1_cas_saleOfInvestmentProperties,1_cas_purchaseOfInvestmentProperties,1_cas_netInvestmentPropertiesPurchaseAndSale,1_cas_saleOfIntangibles,1_cas_purchaseOfIntangibles,1_cas_netIntangiblesPurchaseAndSale,1_cas_netOtherInvestingChanges,1_cas_cashFlowInvestingActivities,1_cas_netIssuancePaymentsOfDebt,1_cas_netLongTermDebtIssuance,1_cas_netShortTermDebtIssuance,1_cas_commonStockDividendPaid,1_cas_preferredStockDividendPaid,1_cas_cash

In [58]:
# Step 1 - Test - Test - Spit
X = df.drop('label',axis=1)
y = df['label']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=99)


In [60]:
X_train.head()

,industry,0_inc_totalRevenue,0_inc_costOfRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netIncomeContinuousOperations,0_inc_netInterestIncome,0_inc_interestIncome,0_inc_otherIncomeExpense,0_inc_operatingExpense,0_inc_totalExpenses,0_inc_taxProvision,0_inc_interestExpense,0_inc_SGA,0_inc_researchAndDevelopment,0_inc_ebit,0_inc_dilutedEPS,0_inc_basicEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_currentAssets,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_financialAssets,0_bal_goodwill,0_bal_goodwillAndOtherIntangibleAssets,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_currentDebt,0_bal_payablesAndAccruedExpenses,0_bal_otherCurrentLiabilities,0_bal_currentLiabilities,0_bal_longTermDebt,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_totalNonCurrentLiabilities,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_cas_netIncome,0_cas_netIncomeFromContinuingOperations,0_cas_depreciation,0_cas_stockBasedCompensation,0_cas_cashFlowOperatingActivities,0_cas_capitalExpenditure,0_cas_saleOfBusiness,0_cas_purchaseOfBusiness,0_cas_netBusinessPurchaseAndSale,0_cas_saleOfPPE,0_cas_purchaseOfPPE,0_cas_netPPEPurchaseAndSale,0_cas_saleOfInvestment,0_cas_purchaseOfInvestment,0_cas_netInvestmentPurchaseAndSale,0_cas_saleOfInvestmentProperties,0_cas_purchaseOfInvestmentProperties,0_cas_netInvestmentPropertiesPurchaseAndSale,0_cas_saleOfIntangibles,0_cas_purchaseOfIntangibles,0_cas_netIntangiblesPurchaseAndSale,0_cas_netOtherInvestingChanges,0_cas_cashFlowInvestingActivities,0_cas_netIssuancePaymentsOfDebt,0_cas_netLongTermDebtIssuance,0_cas_netShortTermDebtIssuance,0_cas_commonStockDividendPaid,0_cas_preferredStockDividendPaid,0_cas_cashDividendsPaid,0_cas_netCommonStockIssuance,0_cas_netPreferredStockIssuance,0_cas_repurchaseOfCapitalStock,0_cas_netOtherFinancingCharges,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_totalRevenue,1_inc_costOfRevenue,1_inc_grossProfit,1_inc_netIncome,1_inc_operatingIncome,1_inc_netIncomeContinuousOperations,1_inc_netInterestIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_operatingExpense,1_inc_totalExpenses,1_inc_taxProvision,1_inc_interestExpense,1_inc_SGA,1_inc_researchAndDevelopment,1_inc_ebit,1_inc_dilutedEPS,1_inc_basicEPS,1_bal_cashEquivalent,1_bal_receivables,1_bal_finishedGoods,1_bal_workInProcess,1_bal_rawMaterials,1_bal_otherCurrentAssets,1_bal_inventory,1_bal_currentAssets,1_bal_netPPE,1_bal_otherNonCurrentAssets,1_bal_financialAssets,1_bal_goodwill,1_bal_goodwillAndOtherIntangibleAssets,1_bal_otherIntangibleAssets,1_bal_nonCurrentAccountsReceivable,1_bal_totalNonCurrentAssets,1_bal_currentDebt,1_bal_payablesAndAccruedExpenses,1_bal_otherCurrentLiabilities,1_bal_currentLiabilities,1_bal_longTermDebt,1_bal_otherNonCurrentLiabilities,1_bal_nonCurrentDeferredLiabilities,1_bal_nonCurrentDeferredTaxesLiabilities,1_bal_longTermProvisions,1_bal_totalNonCurrentLiabilities,1_bal_retainedEarnings,1_bal_stockholdersEquity,1_cas_netIncome,1_cas_netIncomeFromContinuingOperations,1_cas_depreciation,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_capitalExpenditure,1_cas_saleOfBusiness,1_cas_purchaseOfBusiness,1_cas_netBusinessPurchaseAndSale,1_cas_saleOfPPE,1_cas_purchaseOfPPE,1_cas_netPPEPurchaseAndSale,1_cas_saleOfInvestment,1_cas_purchaseOfInvestment,1_cas_netInvestmentPurchaseAndSale,1_cas_saleOfInvestmentProperties,1_cas_purchaseOfInvestmentProperties,1_cas_netInvestmentPropertiesPurchaseAndSale,1_cas_saleOfIntangibles,1_cas_purchaseOfIntangibles,1_cas_netIntangiblesPurchaseAndSale,1_cas_netOtherInvestingChanges,1_cas_cashFlowInvestingActivities,1_cas_netIssuancePaymentsOfDebt,1_cas_netLongTermDebtIssuance,1_cas_netShortTermDebtIssuance,1_cas_commonStockDividendPaid,1_cas_preferredStockDividendPaid,1_cas_cash

In [159]:
# Imputation transformer

trf1 = ColumnTransformer([
    ('Impute_age', SimpleImputer(strategy='constant',fill_value=0),slice(1,426)),
     ],remainder='passthrough')

In [88]:
# One hot encoding - Apply to first column (Industry)

trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False, handle_unknown='ignore'),[0]),
       ],remainder='passthrough')

In [116]:
# Scaling 
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,426)),
       ],remainder='passthrough')

In [198]:
# Resample

trf4 = SMOTE()

In [172]:
# Feature selection

trf5 = SelectKBest(score_func=chi2,k=50)

In [192]:
# train the model
trf6 = xgb.XGBClassifier(random_state=666)

In [199]:
from imblearn.pipeline import Pipeline
# Create Pipeline

pipe = Pipeline([
    ('trf1',trf2),
    ('trf2',trf1),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

In [196]:
X_train.head()

,industry,0_inc_totalRevenue,0_inc_costOfRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netIncomeContinuousOperations,0_inc_netInterestIncome,0_inc_interestIncome,0_inc_otherIncomeExpense,0_inc_operatingExpense,0_inc_totalExpenses,0_inc_taxProvision,0_inc_interestExpense,0_inc_SGA,0_inc_researchAndDevelopment,0_inc_ebit,0_inc_dilutedEPS,0_inc_basicEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_currentAssets,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_financialAssets,0_bal_goodwill,0_bal_goodwillAndOtherIntangibleAssets,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_currentDebt,0_bal_payablesAndAccruedExpenses,0_bal_otherCurrentLiabilities,0_bal_currentLiabilities,0_bal_longTermDebt,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_totalNonCurrentLiabilities,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_cas_netIncome,0_cas_netIncomeFromContinuingOperations,0_cas_depreciation,0_cas_stockBasedCompensation,0_cas_cashFlowOperatingActivities,0_cas_capitalExpenditure,0_cas_saleOfBusiness,0_cas_purchaseOfBusiness,0_cas_netBusinessPurchaseAndSale,0_cas_saleOfPPE,0_cas_purchaseOfPPE,0_cas_netPPEPurchaseAndSale,0_cas_saleOfInvestment,0_cas_purchaseOfInvestment,0_cas_netInvestmentPurchaseAndSale,0_cas_saleOfInvestmentProperties,0_cas_purchaseOfInvestmentProperties,0_cas_netInvestmentPropertiesPurchaseAndSale,0_cas_saleOfIntangibles,0_cas_purchaseOfIntangibles,0_cas_netIntangiblesPurchaseAndSale,0_cas_netOtherInvestingChanges,0_cas_cashFlowInvestingActivities,0_cas_netIssuancePaymentsOfDebt,0_cas_netLongTermDebtIssuance,0_cas_netShortTermDebtIssuance,0_cas_commonStockDividendPaid,0_cas_preferredStockDividendPaid,0_cas_cashDividendsPaid,0_cas_netCommonStockIssuance,0_cas_netPreferredStockIssuance,0_cas_repurchaseOfCapitalStock,0_cas_netOtherFinancingCharges,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_totalRevenue,1_inc_costOfRevenue,1_inc_grossProfit,1_inc_netIncome,1_inc_operatingIncome,1_inc_netIncomeContinuousOperations,1_inc_netInterestIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_operatingExpense,1_inc_totalExpenses,1_inc_taxProvision,1_inc_interestExpense,1_inc_SGA,1_inc_researchAndDevelopment,1_inc_ebit,1_inc_dilutedEPS,1_inc_basicEPS,1_bal_cashEquivalent,1_bal_receivables,1_bal_finishedGoods,1_bal_workInProcess,1_bal_rawMaterials,1_bal_otherCurrentAssets,1_bal_inventory,1_bal_currentAssets,1_bal_netPPE,1_bal_otherNonCurrentAssets,1_bal_financialAssets,1_bal_goodwill,1_bal_goodwillAndOtherIntangibleAssets,1_bal_otherIntangibleAssets,1_bal_nonCurrentAccountsReceivable,1_bal_totalNonCurrentAssets,1_bal_currentDebt,1_bal_payablesAndAccruedExpenses,1_bal_otherCurrentLiabilities,1_bal_currentLiabilities,1_bal_longTermDebt,1_bal_otherNonCurrentLiabilities,1_bal_nonCurrentDeferredLiabilities,1_bal_nonCurrentDeferredTaxesLiabilities,1_bal_longTermProvisions,1_bal_totalNonCurrentLiabilities,1_bal_retainedEarnings,1_bal_stockholdersEquity,1_cas_netIncome,1_cas_netIncomeFromContinuingOperations,1_cas_depreciation,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_capitalExpenditure,1_cas_saleOfBusiness,1_cas_purchaseOfBusiness,1_cas_netBusinessPurchaseAndSale,1_cas_saleOfPPE,1_cas_purchaseOfPPE,1_cas_netPPEPurchaseAndSale,1_cas_saleOfInvestment,1_cas_purchaseOfInvestment,1_cas_netInvestmentPurchaseAndSale,1_cas_saleOfInvestmentProperties,1_cas_purchaseOfInvestmentProperties,1_cas_netInvestmentPropertiesPurchaseAndSale,1_cas_saleOfIntangibles,1_cas_purchaseOfIntangibles,1_cas_netIntangiblesPurchaseAndSale,1_cas_netOtherInvestingChanges,1_cas_cashFlowInvestingActivities,1_cas_netIssuancePaymentsOfDebt,1_cas_netLongTermDebtIssuance,1_cas_netShortTermDebtIssuance,1_cas_commonStockDividendPaid,1_cas_preferredStockDividendPaid,1_cas_cash

In [201]:
pipe.fit(X,y)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [0])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Impute_age',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant'),
                                                  slice(1, 426, None))])),
                ('trf3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 426,

NameError: name 'RepeatedStratifiedKFold' is not defined

## Pipeline vs make_pipeline

Pipeline requires naming of steps, make_pipeline does not.

Same applies to Column_Transformer vs make_column_transformer)

In [130]:
from sklearn.pipeline import Pipeline, make_pipeline
# Alternate Syntax
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [174]:
# train
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [0])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Impute_age',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant'),
                                                  slice(1, 426, None))])),
                ('trf3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 426,

## Explore Pipeline

In [162]:
pipe.named_steps['trf1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('Impute_age',
                                 SimpleImputer(fill_value=0,
                                               strategy='constant'),
                                 slice(1, 426, None))])

In [175]:
# Predict 
y_pred = pipe.predict(X_test)

In [170]:
x_pred

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2,
       3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 2, 2, 1, 2, 2, 3, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3])

In [176]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.3916083916083916

## Imbalanced Data - Use Smote

## Cross Validation using Pipeline

In [204]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X, y, cv=3, scoring='accuracy').mean()

ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/imblearn/pipeline.py", line 293, in fit
    Xt, yt = self._fit(X, y, **fit_params_steps)
  File "/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/imblearn/pipeline.py", line 250, in _fit
    X, y, fitted_transformer = fit_resample_one_cached(
  File "/home/priyesh/projects/MD7/venv/lib/python3.9/site-packages/joblib/memory.py", line 353, in __call__
    return self.func(*args, **k

In [ ]:
oversample = SMOTE()
undersample = RandomUnderSampler()
#steps = [("o", oversample),("u",undersample)]
steps = [("o", oversample)]
pipeline = Pipeline(steps=steps)
# transform the dataset 
X_encoded, y = pipeline.fit_resample(X_encoded,y)